<a href="https://colab.research.google.com/github/Soham-Dutta2003/PRODIGY_GA_01/blob/main/PRODIGY_GA_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Task_01 : Text Generation with GPT-2**

*Train a model to generate coherent and contextually relevant text based on a given prompt. Starting with GPT-2, a transformer model developed by OpenAI, you will learn how to fine-tune the model on a custom dataset to create text that mimics the style and structure of your training data.*

In [ ]:
from google.colab import drive
drive.mount('/content/drive')





Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install transformers datasets evaluate
!pip install torch # If not already installed

In [ ]:
from datasets import Dataset
import pandas as pd

# Define the path to the data file in Google Drive
drive_path = "/content/drive/MyDrive/data.txt"

# Read the data into a pandas DataFrame
df = pd.read_csv(drive_path, header=None, names=["text"])

# Convert the pandas DataFrame to a Hugging Face Dataset
dataset = Dataset.from_pandas(df)

# Display the dataset
print(dataset)

with open(drive_path, 'r') as file:
    data = file.read()
    print(data)

Dataset({
    features: ['text'],
    num_rows: 2
})
Once upon a time...
The knight rode through the misty forest...



In [ ]:
from transformers import GPT2Tokenizer

In [ ]:
# Load tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

# Tokenize dataset
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=512)

tokenized_dataset = dataset.map(tokenize_function, batched=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

In [ ]:
from transformers import GPT2LMHeadModel

model = GPT2LMHeadModel.from_pretrained("gpt2")
model.resize_token_embeddings(len(tokenizer))


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Embedding(50257, 768)

In [ ]:
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling

training_args = TrainingArguments(
    output_dir="./gpt2-finetuned",
    learning_rate=2e-5,
    per_device_train_batch_size=2,
    num_train_epochs=3,
    weight_decay=0.01,
    save_strategy="epoch",
    logging_dir='./logs',
    logging_steps=10,
)

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=data_collator,
)

In [ ]:
def add_labels(examples):
    examples["labels"] = examples["input_ids"]
    return examples

tokenized_dataset = tokenized_dataset.map(add_labels, batched=True)
print(tokenized_dataset)

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

Dataset({
    features: ['text', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 2
})


In [ ]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [ ]:
trainer.train()


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: ssohamdutta (ssohamdutta-meghnad-saha-institute-of-technology) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss


TrainOutput(global_step=3, training_loss=4.136037190755208, metrics={'train_runtime': 617.311, 'train_samples_per_second': 0.01, 'train_steps_per_second': 0.005, 'total_flos': 1567752192000.0, 'train_loss': 4.136037190755208, 'epoch': 3.0})

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, pipeline

# Load tokenizer and model from local checkpoint
model_path = "/content/gpt2-finetuned/checkpoint-3"  # Ensure this path exists

tokenizer = GPT2Tokenizer.from_pretrained(model_path, local_files_only=True)
model = GPT2LMHeadModel.from_pretrained(model_path, local_files_only=True)

# Create text generation pipeline
text_gen = pipeline("text-generation", model=model, tokenizer=tokenizer)

# Generate text
prompt = "Once upon a time"
generated = text_gen(prompt, max_length=100, num_return_sequences=1)

print(generated[0]["generated_text"])


Device set to use cpu
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Both `max_new_tokens` (=256) and `max_length`(=100) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Once upon a time, the game would be a disaster for anyone who thought to get a hold of it. You would have to have been a good person to buy a game. You would have to be able to make money doing it. But if you had been a good person at all, you would have purchased the game.

That is, even though it's a game that you could have bought at a store and had a good time in without getting banned for it.

However, in my experience, it still wasn't worth it.

I think the people who bought the game were just too lazy. They didn't know how to read the game. Their minds would have been spinning so quickly. I'm sure most people would have understood how easy it would be to get banned.

I don't think that it's just because the people who bought the game weren't good.

It's just that the people who bought the game were just too much of a waste of money.

The only reason why I bought this game was because I wanted to play the game, and would have liked to.

I had no clue what the game was, and I didn